In [ ]:
import numpy as np
import tensorflow as tf
import cv2
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
import os
import zipfile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
e = 42 #indicação do número da parte a retreinar

In [ ]:
zip_ref = zipfile.ZipFile('/content/drive/My Drive/Full_dataset/' + str(e) + '.zip', 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
SIZE = 256
image_dataset = [] 
mask_dataset = []  

In [ ]:
images = os.listdir('/tmp/' + str(e) + '/images/')
for i, image_name in enumerate(images):    
    if (image_name.split('.')[1] == 'tif'):
        print('Encontradas ' + str(len(image_dataset) + 1) + ' imagens')
        image = cv2.imread('/tmp/' + str(e) + '/images/' +image_name, cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_dataset.append(np.array(image))

image_dataset = (tf.keras.utils.normalize(np.array(image_dataset), axis=1))

In [ ]:
masks = os.listdir('/tmp/' + str(e) + '/masks/')
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        print('Encontradas ' + str(len(mask_dataset) + 1) + ' máscaras')
        image = cv2.imread('/tmp/' + str(e) + '/masks/' + image_name, 0)
        mask_dataset.append(np.array(image))

mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.2, random_state = 0)

early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

filepath= '/content/drive/My Drive/Full_dataset/historico1/' + str(e) + '/weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5'
check = tf.keras.callbacks.ModelCheckpoint(filepath, monitor= 'val_accuracy', verbose=1, save_best_only=True, mode='max')

csv = tf.keras.callbacks.CSVLogger('/content/drive/My Drive/Full_dataset/historico1/' + str(e) + '/mylogs.csv' , separator=';', append=False)

callback = [early, check, csv]

model = tf.keras.models.load_model('/content/drive/My Drive/Full_dataset/historico1/' + str(e-1) + '/weights-improvement-06-0.957.hdf5') #indicação do modelo a retreinar

history = model.fit(X_train, y_train, 
                    batch_size = 16, 
                    callbacks = callback,
                    verbose=1, 
                    epochs=1000, 
                    validation_data=(X_test, y_test))